## Solution: 
our solution consists of two parts: special smart contract and backend. 

The key point in contract is that only smart contract will be granted to make transfer, thus, centralized part remains trustless. 

The backend denies all permissions to transfer to wrong address and only proxies special password to the smart contract to perform transfer checking password hash

Backend consists of js blockchain callers (here we use rarible ERC721ABI and rarible wrapper of  "safeTransferFrom" method) and local database to track & match pending gifts. Client part uses pure js code.

In [1]:
import os
import json
import requests

In [20]:
%matplotlib inline
%config Completer.use_jedi = False

In [32]:
# who sends an nft (so, the nft owner and who wants to send a present, i.e. sender)
SENDER_PRIVATE_KEY="
SENDER_ADDRESS="

In [33]:
# the centralized backend
BACK_ADDRESS = "
BACK_PRIVATE = "

In [34]:
# receiver?
MY_ACC_PRIVATE = 
MY_ACC_ADDRESS = 

In [43]:
link = "https://rinkeby.rarible.com/token/0x509fd4cdaa29be7b1fad251d8ea0fca2ca91eb60:111795?tab=details"
numbers = link.split('/')[-1].split(':')
nft_contract, nft_token = numbers[0], numbers[1].split('?')[0]

In [36]:
nft_contract, nft_token

('0x509fd4cdaa29be7b1fad251d8ea0fca2ca91eb60', '111795')

In [45]:
# create contract from a backend, so only this backend can call the one
# after the deploy: save the contract address
data = {
    "debug": "NO",
    "private_ext": BACK_PRIVATE
}
r = requests.post("http://localhost:8001/deploy", json=data)

In [46]:
transfer_contract = json.loads(r.content.decode())["link"]
transfer_contract

'0x0264837577091a402a6A62738a9e5Fecad107407'

In [47]:
# todo: hardcode
# transfer_contract = "0xd4Ba7daA7F3A5DbE2453210C4D458959687C20a6"

## User Flow

In [52]:
# add permission to the contract (and only to the one) call transfer (so permission from nft owner).
# !The sender should be nft owner
# PS. it is possible to combine with the following (but no win?)
data = {
    "debug": "NO",
    "private_ext": SENDER_PRIVATE_KEY,
    "owner_address": transfer_contract,
    "nft_contract": nft_contract,
    "nft_token": nft_token,
}
r = requests.post("http://localhost:8001/permission", json=data)
r, r.content

(<Response [200]>, b'{"status":"OK"}')

In [53]:
# set up pasword hash stright into contarct
# so, the password may know only sender and receiver: no need to believe to our back-end
sender_password = "TEST_PASSWORD"

data = {
    "transfer_contract": transfer_contract,
    "password": sender_password,
    "nft_contract": nft_contract,
    "token": nft_token,
    "private_ext": SENDER_PRIVATE_KEY
}
r = requests.post("http://localhost:8001/setpassword", json=data)
r, r.content

(<Response [200]>, b'{"status":"OK"}')

In [54]:
# book a gift for a receiver.
# We do it centralized with our back-end, so backend validate that receiver is preferable that sender wanted to (e.g. with help of created password)
# We send money (per gas) to the validated receiver (to his public address)
data = {
    "transfer_contract": transfer_contract,
    "receiver": MY_ACC_ADDRESS,
    "nft_contract": nft_contract,
    "token": nft_token,
    "private_ext": BACK_PRIVATE
}
r = requests.post("http://localhost:8001/book", json=data)
r, r.content

(<Response [200]>, b'{"status":"OK"}')

In [55]:
# do the transfer to receiver address.
# It may be called local from receiver frontend coz we sent him money so he able to proceed gas payment
data = {
    "transfer_contract": transfer_contract,
    "receiver": MY_ACC_ADDRESS,
    "sender": SENDER_ADDRESS,
    "nft_contract": nft_contract,
    "token": nft_token,
    "password": sender_password,
    "private_ext": MY_ACC_PRIVATE
}
r = requests.post("http://localhost:8001/dtransfer", json=data)
r, r.content

(<Response [200]>, b'{"status":"OK"}')